In [73]:
import tensorflow as tf
import pandas as pd
import numpy as np
import wandb

# Tensorflow

#### Config

In [74]:

config = {
    "learning_rate": 1e-2,
    "epochs": 50,
    "batch_size": 64,
    "optimizer": "Adam",
    "dropout_rate": 0.3
}

### WandB connection

In [ ]:

wandb.login()
wandb.init(project="assignment-1", name="tensorflow-32-16", reinit=True, config=config)

### Loading the data

In [76]:
x_train = pd.read_csv('data/train/data.csv').to_numpy(dtype='float32')
y_train = pd.read_csv('data/train/labels.csv').to_numpy(dtype='float32')

x_val = pd.read_csv('data/val/data.csv').to_numpy(dtype='float32')
y_val = pd.read_csv('data/val/labels.csv').to_numpy(dtype='float32')

print(x_train[0])
print(y_train[0])
print()
print(x_val[0])
print(y_val[0])

[2.         1.         0.33240056 1.         1.         0.58483976]
[1.]

[ 3.          0.         -0.67140484  0.          0.         -0.81169564]
[0.]


### Building a model

In [77]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(6,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(rate=config['dropout_rate']),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),    
])

In [78]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 32)                224       
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_19 (Dense)            (None, 16)                528       
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_20 (Dense)            (None, 8)                 136       
                                                                 
 dense_21 (Dense)            (None, 1)                 9         
                                                                 
Total params: 897
Trainable params: 897
Non-trainable 

In [79]:
if config['optimizer'] == 'SGD':
    optimizer = tf.keras.optimizers.SGD(learning_rate=config['learning_rate'])
elif config['optimizer'] == 'Adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'], beta_1=0.9, beta_2=0.99)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [80]:
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        #return 1 / (1 + config['decay_rate'] * epoch) * config['learning_rate']\n",
        return lr * 0.82

scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

### Training the model

In [81]:
history=model.fit(
    x_train,
    y_train,
    epochs=config['epochs'],
    validation_data=(x_val, y_val),
    batch_size=config['batch_size'],
    shuffle=True,
    callbacks=[
        scheduler_callback,
        wandb.keras.WandbCallback()
    ]
)

Epoch 1/50
 1/10 [==>...........................] - ETA: 11s - loss: 0.7446 - accuracy: 0.4062WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0026s vs `on_train_batch_end` time: 0.0030s). Check your callbacks.
INFO:tensorflow:Assets written to: c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230330_141912-hna4uem0\files\model-best\assets


wandb: Adding directory to artifact (c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230330_141912-hna4uem0\files\model-best)... Done. 0.0s


10/10 [==============================] - 5s 376ms/step - loss: 0.6721 - accuracy: 0.5948 - val_loss: 0.6137 - val_accuracy: 0.6604 - lr: 0.0100
Epoch 2/50
 1/10 [==>...........................] - ETA: 0s - loss: 0.6555 - accuracy: 0.6719INFO:tensorflow:Assets written to: c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230330_141912-hna4uem0\files\model-best\assets


wandb: Adding directory to artifact (c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230330_141912-hna4uem0\files\model-best)... Done. 0.0s


10/10 [==============================] - 3s 337ms/step - loss: 0.6106 - accuracy: 0.6759 - val_loss: 0.5551 - val_accuracy: 0.7245 - lr: 0.0100
Epoch 3/50
 1/10 [==>...........................] - ETA: 0s - loss: 0.6091 - accuracy: 0.6094INFO:tensorflow:Assets written to: c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230330_141912-hna4uem0\files\model-best\assets


...

10/10 [==============================] - 3s 294ms/step - loss: 0.3954 - accuracy: 0.8298 - val_loss: 0.4214 - val_accuracy: 0.8264 - lr: 9.2420e-04
Epoch 23/50
 3/10 [========>.....................] - ETA: 0s - loss: 0.4446 - accuracy: 0.8177INFO:tensorflow:Assets written to: c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230330_141912-hna4uem0\files\model-best\assets


wandb: Adding directory to artifact (c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230330_141912-hna4uem0\files\model-best)... Done. 0.0s


10/10 [==============================] - 3s 337ms/step - loss: 0.4177 - accuracy: 0.8298 - val_loss: 0.4207 - val_accuracy: 0.8340 - lr: 7.5784e-04
Epoch 24/50
 1/10 [==>...........................] - ETA: 0s - loss: 0.4073 - accuracy: 0.7969INFO:tensorflow:Assets written to: c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230330_141912-hna4uem0\files\model-best\assets


wandb: Adding directory to artifact (c:\Users\filip\Desktop\NS\assignment-1\wandb\run-20230330_141912-hna4uem0\files\model-best)... Done. 0.0s


10/10 [==============================] - 3s 310ms/step - loss: 0.4159 - accuracy: 0.8233 - val_loss: 0.4200 - val_accuracy: 0.8340 - lr: 6.2143e-04
Epoch 25/50
10/10 [==============================] - 0s 10ms/step - loss: 0.4071 - accuracy: 0.8282 - val_loss: 0.4201 - val_accuracy: 0.8302 - lr: 5.0957e-04
Epoch 26/50
10/10 [==============================] - 0s 10ms/step - loss: 0.4147 - accuracy: 0.8201 - val_loss: 0.4202 - val_accuracy: 0.8302 - lr: 4.1785e-04
Epoch 27/50
10/10 [==============================] - 0s 11ms/step - loss: 0.4085 - accuracy: 0.8363 - val_loss: 0.4204 - val_accuracy: 0.8302 - lr: 3.4264e-04
Epoch 28/50
10/10 [==============================] - 0s 11ms/step - loss: 0.4076 - accuracy: 0.8201 - val_loss: 0.4205 - val_accuracy: 0.8302 - lr: 2.8096e-04
Epoch 29/50
10/10 [==============================] - 0s 11ms/step - loss: 0.4234 - accuracy: 0.8120 - val_loss: 0.4205 - val_accuracy: 0.8302 - lr: 2.3039e-04
Epoch 30/50
10/10 [==============================] - 0s 1

In [82]:
import plotly.express as px
fig = px.line({'loss': history.history['loss'], 'val_loss': history.history['val_loss']})
fig.show()

In [83]:
import plotly.express as px
fig = px.line({'accuracy': history.history['accuracy'], 'val_accuracy': history.history['val_accuracy']})
fig.show()

In [84]:
wandb.finish()

accuracy,▁▃▅▅▇▇▇▆█▇▇▇█▇▇▇▇██▇▇█▇▇▇▇▇▇█▇█████▇▇▇█▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▅▄▂▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▂▁▂▁▂▂▂▁▁▁▁▁▁▂▁▁▁
lr,████████▇▆▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅▆▇▇▇▇██▇▇████████████████████████████
val_loss,█▆▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.82658
best_epoch,23
best_val_loss,0.42003
epoch,49
loss,0.39504
